In [1]:
import scqubits as scq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import e,h,pi
phi0 = 2.0678338484619295e-15 # flux quantum

### choose an inductance, $L_J$, and calculate $E_J = \frac{\Phi_0^2}{4\pi^2L_J}$ for both qubits

In [2]:
## choose LJ ##
LJ = 10e-9 # 10 nH 
EJ = phi0**2/((2*np.pi)**2*LJ)
EJ_MHz = EJ / h * 1e-6 # convert to MHz

# Pocket transmon

In [3]:
pocket_df = pd.read_csv("pocket_electro_output/terminal-C.csv",usecols =[1,2,3]) # dropped palace index labeling

In [4]:
## let's take a look at our capacitance matrix
pocket_df

,C[i][1] (F),C[i][2] (F),C[i][3] (F)
0,1.107092e-13,-4.262233e-14,-1.352754e-14
1,-4.262233e-14,1.038633e-13,-1.351485e-15
2,-1.352754e-14,-1.351485e-15,2.795796e-14


### extract $C_\Sigma$ and calculate $E_C = \frac{e^2}{2C_\Sigma}$

in the config file we gave the top capacitor pad index 1 and the bottom capacitor pad index 2, so their corresponding matrix elements are:

$C_{top} = C_{11}$

$C_{bottom} = C_{22}$

$C_{mutal} = |C_{12}| = |C_{21}|$  

The total shunt capacitance is given by: $C_\Sigma = |C_{12}| + \frac{(C_{11} + C_{12})(C_{22} + C_{21})}{C_{11} + C_{22} + 2C_{12}}$ (See [this article](https://arxiv.org/pdf/2402.04322))

Index 3 corresponds to the qubit-resonator coupler - we won't use this for now.

In [5]:
C11 = pocket_df.iloc[0][0] # self-capacitance of top pad
C22 = pocket_df.iloc[1][1] # self-capacitance of bottom pad
C12 = pocket_df.iloc[0][1] # negative of mutal capacitance between pads 

## total shunt capacitance ##
C_sigma_pocket = abs(C12) + ((C11 + C12)* (C22 + C12)) / (C11 + C22 + 2*C12)
print("Total Shunt Capacitance = {} fF".format(np.round(C_sigma_pocket*1e15,2)))

## calculate qubit charging energy 
EC_pocket = e**2/(2*C_sigma_pocket) 
EC_pocket_MHz = EC_pocket / h * 1e-6 # convert to MHz
print("EC = {} MHz".format(np.round(EC_pocket_MHz,2)))

Total Shunt Capacitance = 74.86 fF
EC = 258.74 MHz


### extract Hamiltonian parameters with scQubits

In [6]:
pocket_transmon = scq.Transmon(EJ=EJ_MHz,
                              EC=EC_pocket_MHz,
                              ng=0,
                              ncut=31)

f_q_pocket = pocket_transmon.E01() / 1000 # in GHz
alpha_pocket = pocket_transmon.anharmonicity() # in MHz

print("=====================================")
print("Hamitlonian Parameters:")
print("Qubit Frequency = {} GHz".format(np.round(f_q_pocket,2)))
print("Qubit Anharmonicity = {} MHz".format(np.round(alpha_pocket,2)))
print("=====================================")

Hamitlonian Parameters:
Qubit Frequency = 5.54 GHz
Qubit Anharmonicity = -291.69 MHz


# Xmon

In [7]:
xmon_df = pd.read_csv("xmon_electro_output/terminal-C.csv",usecols =[1,2]) # dropped palace index labeling

In [8]:
## let's take a look at our capacitance matrix
xmon_df

,C[i][1] (F),C[i][2] (F)
0,1.092349e-13,-8.975294e-16
1,-8.975294e-16,3.267345e-14


### extract $C_\Sigma$ and calculate $E_C = \frac{e^2}{2C_\Sigma}$

Since this is a grounded qubit (xmon style) extracting the qubit capacitance is more straightforward. All we need is the capacitance of the qubit conductor cross which we gave index 1 in the config file so:

$C_\Sigma = C_{cross} = C_{11}$

Index 2 corresponds to the qubit-resonator coupler - we won't use this for now.

In [9]:
## total shunt capacitance ##
C_sigma_xmon = xmon_df.iloc[0][0]
print("Total Shunt Capacitance = {} fF".format(np.round(C_sigma_xmon*1e15,2)))

## calculate qubit charging energy 
EC_xmon = e**2/(2*C_sigma_xmon) 
EC_xmon_MHz = EC_xmon / h * 1e-6 
print("EC = {} MHz".format(np.round(EC_xmon_MHz,2)))

Total Shunt Capacitance = 109.23 fF
EC = 177.33 MHz


### extract Hamiltonian parameters with scQubits

In [10]:
xmon = scq.Transmon(EJ=EJ_MHz,
                    EC=EC_xmon_MHz,
                    ng=0,
                    ncut=31)

f_q_xmon = xmon.E01() / 1000 # in GHz
alpha_xmon = xmon.anharmonicity() # in MHz

print("=====================================")
print("Hamitlonian Parameters:")
print("Qubit Frequency = {} GHz".format(np.round(f_q_xmon,2)))
print("Qubit Anharmonicity = {} MHz".format(np.round(alpha_xmon,2)))
print("=====================================")

Hamitlonian Parameters:
Qubit Frequency = 4.63 GHz
Qubit Anharmonicity = -195.12 MHz
